In [174]:
from bs4 import BeautifulSoup as bs
import requests
import urllib.request

import model_lib 

hostname = 'https://en.wikipedia.org{}'

def get_wiki(cat):
    to_drop = ['','CS1: long volume value','Categories','Category',
 'Talk',
 'Contributions',
 'Article',
 'Read',
 'Main page',
 'Contents',
 'Current events',
 'Random article',
 'About Wikipedia',
 'Contact us',
 'Donate',
 'Help',
 'Community portal',
 'Recent changes',
 'Upload file',
 'What links here',
 'Related changes',
 'Special pages',
 'Wikidata item',
 'Edit links',
 'Creative Commons Attribution-ShareAlike License',
 'Terms of Use',
 'Privacy Policy',
 'Privacy policy',
 'Disclaimers',
 'Contact Wikipedia',
 'Developers',
 'Cookie statement',
  'Library cataloging',
 'classification']
    res = requests.get(cat)
    soup = bs(res.text, "html.parser")
    category = {}
    articles = {}
    for link in soup.find_all("a"):
        url = link.get("href", "")
        if "/wiki/Category" in url:
            category[link.text.strip()] = url
        if "/wiki/" in url:
            articles[link.text.strip()] = url
    category = {k:hostname.format(category[k]) for k in category.keys() if k not in to_drop and 'CS1' not in k and 'wiki' not in k.lower() and 'wikipedia.org' not in category[k].lower()}
    articles = {k:hostname.format(articles[k]) for k in articles.keys() if k not in to_drop and 'CS1' not in k and 'wiki' not in k.lower() and 'wikipedia.org' not in articles[k].lower() and 'Category:' not in articles[k]}
    return category, articles

def articles_content(url):
    import re
    def striphtml(data):
        p = re.compile(r'<.*?>')
        return p.sub('', data)
    page = urllib.request.urlopen(url)
    soup = bs(page, "lxml")
    content = soup.find_all('p')



    output = ''
    blacklist = [
        '[document]',
        'noscript',
        'header',
        'html',
        'meta',
        'head', 
        'input',
        'script',
        # there may be more elements you don't want, such as "style", etc.
    ]

    for t in content:
        if t.parent.name not in blacklist:
            output += '{} '.format(t)

    return striphtml(output)

In [138]:
list_categories = ['https://en.wikipedia.org/wiki/Category:Agriculture','https://en.wikipedia.org/wiki/Nutrient_management']
categories, articles = {},{}
for e in list_categories:
    cat, art = get_wiki(e)
    categories = {**categories,**cat}
    articles = {**articles,**art}
print(len(articles))
url = articles['Agriculture']
print(url)
articles_content(url)

198


In [143]:
target_categorie = ['Agriculture', 'Energy', 'Health']
list_categories = ['https://en.wikipedia.org/wiki/Category:{}'.format(ctgr) for ctgr in target_categorie]
cat, art = get_wiki(list_categories[0])

In [148]:
'https://en.wikipedia.org/wiki/Category:Agricultural_finance' in list(cat.keys())

False

In [160]:
def update_wiki_terms(target, url):
    dict_links[target].append(url)
    
actors_agriculture = ['productors', 'market', 'investment', 'logistic', 'transformation']
dict_links = {e: [] for e in actors_agriculture}
update_wiki_terms('productors', 'https://en.wikipedia.org/wiki/Farmer')

In [161]:
dict_links['productors']

['https://en.wikipedia.org/wiki/Farmer']

In [202]:
url = 'https://en.wikipedia.org/wiki/Agrarian_society'
_, articles = get_wiki(url)
print(list(articles.keys()))

print(len(articles))
corpus = [articles_content(articles[a]) for a in  articles]

lemm_corpus = [model_lib.Lemm(X = art, file = 'stopwords_agri.txt').X for art in corpus]



from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

tfid_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2)
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2)
tfid = tfid_vectorizer.fit_transform(lemm_corpus)
tf = tf_vectorizer.fit_transform(lemm_corpus)
tfid_feature_name = tfid_vectorizer.get_feature_names()
tf_feature_name = tf_vectorizer.get_feature_names()

print(sorted([a for a in zip(tf_feature_name,tf.tocoo().data)], key = lambda x: x[1], reverse = True)[:5])
print(sorted([a for a in zip(tfid_feature_name,tfid.tocoo().data)], key = lambda x: x[1], reverse = True)[:5])

['History', 'History of organic farming', 'Neolithic Revolution', 'Agriculture in Mesoamerica', 'Austronesian Expansion', 'Ancient Egyptian agriculture', 'Agriculture in ancient Greece', 'Agriculture in ancient Rome', 'Arab Agricultural Revolution', 'British Agricultural Revolution', 'Green Revolution', 'Chinampa', 'Monoculture', 'On land', 'Animal husbandry', 'cattle', 'pigs', 'poultry', 'sheep', 'Dairy', 'Dryland', 'Extensive', 'Fertilizer', 'Free-range', 'Grazing', 'Convertible husbandry', 'Rotational grazing', 'Hobby', 'Intensive', 'animals', 'crops', 'Natural', 'Orchard', 'Organic', 'Paddy field', 'Ranching', 'Sharecropping', 'Slash-and-burn', 'Terrace', 'Steam sterilization', 'Hydroculture', 'Aquaculture', 'Aquaponics', 'Hydroponics', 'Aeroponics', 'Agribusiness', 'Agricultural engineering', 'Agricultural machinery', 'Agricultural science', 'Agricultural technology', 'Agroecology', 'Agroforestry', 'Agronomy', 'Animal-free', 'Crop diversity', 'Ecology', 'Mechanisation', 'Permacult

In [ ]:
url = 'https://en.wikipedia.org/wiki/Agrarian_society'
_, articles = get_wiki(url)
print(list(articles.keys()))

print(len(articles))
corpus = [articles_content(articles[a]) for a in  articles]

lemm_corpus = [model_lib.Lemm(X = art, file = 'stopwords_agri.txt').X for art in corpus]



from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

tfid_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2)
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2)
tfid = tfid_vectorizer.fit_transform(lemm_corpus)
tf = tf_vectorizer.fit_transform(lemm_corpus)
tfid_feature_name = tfid_vectorizer.get_feature_names()
tf_feature_name = tf_vectorizer.get_feature_names()

print(sorted([a for a in zip(tf_feature_name,tf.tocoo().data)], key = lambda x: x[1], reverse = True)[:5])
print(sorted([a for a in zip(tfid_feature_name,tfid.tocoo().data)], key = lambda x: x[1], reverse = True)[:5])

In [229]:
import chalenge1000
Chalenge = chalenge1000.Native()
df = Chalenge.X
corpus = df[df['categorie'] == 'agriculture'][['nom_struc', 'categorie']].join(Chalenge.descriptions_trad)[['prez_struc', 'prez_produit_struc']].fillna('').agg(sum, axis = 1).values.tolist()

lemm_corpus = [model_lib.Lemm(X = art, file = 'stopwords_agri.txt').X for art in corpus]

In [234]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

tfid_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, vocabulary = vocab)
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, vocabulary = vocab)
tfid = tfid_vectorizer.fit_transform(lemm_corpus)
tf = tf_vectorizer.fit_transform(lemm_corpus)
tfid_feature_name = tfid_vectorizer.get_feature_names()
tf_feature_name = tf_vectorizer.get_feature_names()
print(vocab)
print(collections.Counter(' '.join(lemm_corpus).split()).most_common(10))
print(sorted([a for a in zip(tf_feature_name,tf.tocoo().data)], key = lambda x: x[1], reverse = False))
print(sorted([a for a in zip(tfid_feature_name,tfid.tocoo().data)], key = lambda x: x[1], reverse = False))

['product', 'farmer', 'production', 'food', 'agricultural', 'market', 'farm', 'produce', 'service', 'local']
[('product', 1264), ('farmer', 767), ('production', 740), ('food', 726), ('agricultural', 622), ('market', 609), ('farm', 555), ('produce', 507), ('service', 394), ('local', 374)]
[('farmer', 1), ('production', 1), ('produce', 1), ('service', 1), ('local', 1), ('product', 2), ('food', 2), ('agricultural', 3), ('market', 3), ('farm', 9)]
[('market', 0.09666378190669922), ('agricultural', 0.11658972476503757), ('service', 0.22664184546291313), ('produce', 0.2812619214967012), ('production', 0.31418096826340136), ('farmer', 0.3285048893111314), ('food', 0.4910943413136987), ('local', 0.6502495878823172), ('product', 0.7431024187885659), ('farm', 0.9201022523327803)]


In [230]:
[a for a in zip(tf_feature_name,tf.tocoo().data) if 'product' in a[0]]

[('product', 2), ('production', 1)]

In [231]:
[a for a in zip(tfid_feature_name,tfid.tocoo().data) if 'product' in a[0]]

[('product', 0.7431024187885659), ('production', 0.31418096826340136)]

In [ ]:
import collections
vocab = [a for a,b in collections.Counter(' '.join(lemm_corpus).split()).most_common(10)]